In [4]:
import xml.etree.ElementTree as ET

# Variables
name = "admin"
password = "password"

# Create the root element
root = ET.Element("userRegistry")
root.set("xmlns", "http://www.geoserver.org/security/users")
root.set("version", "1.0")

# Create the users element
users = ET.SubElement(root, "users")

# Create the user element
user = ET.SubElement(users, "user")
user.set("enabled", "true")
user.set("name", name)
user.set("password", password)

# Create the groups element
groups = ET.SubElement(root, "groups")

# Create the XML string
xml_string = ET.tostring(root, encoding="utf-8").decode()

# Print the XML output
print(xml_string)


<userRegistry xmlns="http://www.geoserver.org/security/users" version="1.0"><users><user enabled="true" name="user" password="password" /></users><groups /></userRegistry>


In [3]:
import utils
import core

config = '../inputs/config.yml'

configuration = utils.read_configuration(config)
print(configuration)

core.create_local_folders_and_files(config)
core.create_geoserver_user_credentials(config)


{'base_path': '../outputs', 'users': ['admin', 'alice', 'bob'], 'paths': {'secret_path': '../outputs/secrets'}, 'stored_secrets': '../outputs/secrets/secrets.txt'}
INFO: Path '../outputs/secrets' already exists.
INFO: secret.txt already exists.
INFO: Credentials for alice have been updated.
INFO: Credentials for bob have been updated.
INFO: Credentials for bob have been updated.


[('admin',
  'sUOwxaZCVrsIHmoLE5GsE5BqcyFRJKN6eAUwWUCtCeoYovtHHa0229ZTMoYe2mHR',
  'i2SkyCek0IZs*y=S9AeI'),
 ('alice',
  'uf9qQ2QYf5MU5BvitWaehWD6Q1YLmutpsiIN65XystRxNYQe5HNWs6oQGtBYb4Pp',
  '*|KMcj@X6z@9iidp)#2k'),
 ('bob',
  'ikUCpyr5mtCIzNaK5TsI5Kv5Li8D2NenxSibXKIf1yTWiG/qYEsha+QcZcznWL18',
  '=UmcZx.Znh$<~.JU1jNW')]